In [1]:
# 생성자 모델: 가상의 이미지를 생성하는 모델
             
# 판별자 모델: 이미지가 진짜 가짜인지 판별하는 모델 

# gen 모델 : 생성자와 판별자가 결합된 모델
#            생성자가 생성한 이미지를 판별자에게 판별하도록 하는 모델
#            생성자가 생성한 이미지가 True라벨로 학습 => 생성자가 진짜이미지와 같은 이미지를 생성하도록 학습

In [ ]:
# 생성자와 판별자가 서로 대립하며 서로의 성능을 점차 개선해 나가는 쪽으로 학습
# 데이터 증폭(부족한 데이터 보충)
# https://www.casualganpapers.com/image-editing-stylegan2-encoder-generator-tuning-inversion/HyperStyle-explained.html
# https://talkingaboutme.tistory.com/entry/Book-DL-Generative-Deep-Learning

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import os
if not os.path.exists('./gan_images'):
    os.mkdir('./gen_images')

In [6]:
# 생성자

generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))  # 100차원짜리를 받아서 dense 통과
generator.add(BatchNormalization())     # batch단위로 정규화
generator.add(Reshape((7,7,128)))  # 128depth  7,7 이미지
generator.add(UpSampling2D())  # 사이즈커짐 14,14 128로 바뀜
generator.add(Conv2D(64, kernel_size=5, padding='same')) # 14,14 * 64
generator.add(BatchNormalization())  # 데이터값이 바뀌면서 스케일차를 줄여줌
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())  # 14*14 64에서 28,28 64 가 됨
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))  # 28,28,1  (흑백이미지 한장과 같음)
generator.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 6272)              633472    
                                                                 
 batch_normalization_1 (Batc  (None, 6272)             25088     
 hNormalization)                                                 
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_2 (Batc  (None, 14, 14, 64)      

In [13]:
# 판별자

discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))  # CNN판별
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation_1 (Activation)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_2 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [15]:
ginput = Input(shape=(100,))       # input layer            ## gan의 구조 
dis_output = discriminator(generator(ginput))   # output layer
gan = Model(ginput, dis_output) # gan model을 만듬

In [16]:
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_1 (Sequential)   (None, 28, 28, 1)         865281    
                                                                 
 sequential_2 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 1,065,474
Non-trainable params: 12,672
_________________________________________________________________


In [27]:
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [28]:
def gan_train(epoch, batch_size, saving_interval):
    
    (x_train, _),(_, _) = mnist.load_data()  # mnist 이미지 가져옴
    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32')   # reshape
    x_train = (x_train - 127.5)/127.5 # 정규화해줌
    
    true = np.ones((batch_size, 1))  # vector 1 값  1로 채워진 batch_size 정답값
    fake = np.zeros((batch_size, 1)) # 0 값 vector
    
    for i in range(epoch):
        
        idx = np.random.randint(0, x_train.shape[0], batch_size)  # 랜덤하게 배치사이즈 만큼 index 가져오기
        img = x_train[idx]
        d_loss_real = discriminator.train_on_batch(img, true)  # 이것은 진짜 이미지다 학습
        
        noise = np.random.normal(0,1, (batch_size, 100))  # 가우시안분포로 noise 넣어줌
        get_imgs = generator.predict(noise) # 가짜값을 만듬
        d_loss_fake = discriminator.train_on_batch(gen_images, fake) # 이건 페이크다라고 판별자가 학습
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  # 잘 판별할수록 판별자의 d_loss 감소 
        # loss값        
        g_loss = gan.train_on_batch(noise, true) # 가짜벡터주고 진짜야라고 배치단위로 학습
        
        print('epoch : ',i,'d_loss : ', d_loss, 'g_loss : ', g_loss)
        
        if i % saving_interval == 0:  # saving_interval 간격으로 이미지 저장
            noise = np.random.normal(0.1,(25,100))
            gen_imgs = generator.predict(noise)
            
            gen_imgs = 0.5* gen_imgs + 0.5
            
            fig, axs = plt.subplot(5,5)
            count = 0
            
            for i in range(5):
                for k in range(5):
                    axs[i, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                    axs[i, k].axis('off')
                    count += 1
                    
            fig.saving_interval('gan_images/gan_mnist_%d.png'%i)
                    
                    

In [29]:
gan_train(4001, 32, 200)

NameError: name 'images' is not defined